## Overview
Data source: https://openlibrary.org/developers/dumps
The data dumps from openlibrary.org are huge so to make the project manageable, we limited the content to books on articial intelligence. To accomplish this, all three dump files (ol_dump_works_2019_01-31.txt, ol_dump_editions_2019-01-31.txt, and ol_dump_editions_2019-01-31.txt were downloaded from openlibrary.org, then processed as follows:

```
grep -i 'artificial\ intelligence' ol_dump_works_2019_01-31.txt > artificial_intelligence_works.txt
grep -Po 'OL[0-9]+A' artificial_intelligence_works.txt | uniq > ai-authors-id.txt
grep -f ai-authors.txt ol_dump_editions_2019-01-31.txt >  ai-authors.tsv
grep -Po 'OL[0-9]+W' artificial_intelligence_works.txt | uniq > ai-works-id.txt
grep -f ai-works-id.txt  ol_dump_editions_2019-01-31.txt > ai-editions.tsv
```

The code below parses these files and extracts the data needed for the database. 

### Step 1: Process 'works' data
**Inputs:**   
../data_output/artificial_intelligence_works.txt

**Outputs:**   
../data_output/works.csv  
../data_output/author_works.csv  
../data_output/works_subject.csv  
../data_output/subjects.csv  

In [1]:
import csv
import json
import pandas as pd
doc = "../data_output/artificial_intelligence_works.txt"
df = pd.read_csv(doc, sep='\t', header=None)

# add column names
df.columns = ['type', 'path', 'revisions', 'timestamp', 'details']

# get 'work_id' from 'path'
df['work_id'] = df.path.str[7:]

# create a list containing only the id and json data
works = df[['work_id', 'details']].values.tolist()

# initialize some lists to hold the data that will get saved in csv files
titles = [] # work_id and title
work_authors = [] # work_id and author_id
subjects = [] # temporary storage for work_id and subject
subject_tbl = [] # subject_id 
work_subject = [] #
edition_publisher

# Loop through 'works' and pull out the pieces of data we want in the database
for work in works:
    details = json.loads(work[1])
    
    # append title to list
    titles.append([work[0], details["title"]])
    
    # append authors to list - not all works have authors!
    try:
        for a in details["authors"]:
            # work_authors.append([works[0], json.dumps(a["author"]["key"][9:])])
            # author_id = json.dumps(a["author"]["key"][9:])
            work_authors.append([work[0], a["author"]["key"][9:]])
    except:
        # print(json.dumps(details))
        continue
        
    # append subjects to list
    try:
        for s in details["subjects"]:
            subjects.append([work[0], s])
    except:
        # print(json.dumps(details))
        continue

# Create a set from the subjects list  
unique_subjects = set()
for subject in subjects:
    unique_subjects.add(subject[1])
    
# Generate a subject id for each subject in the set.
n = 1
for u in sorted(unique_subjects):
    subject_tbl.append([str(n), u])
    n += 1

# Create a dictionary to allow subject lookups
subject_id = {}
for s in subject_tbl:
    subject_id[s[1]] = s[0] 

# Create a list of lists containing pairs of subject id's and work id's    

for s in subjects:
    work_subject.append([s[0], subject_id[s[1]]])

# Write "titles" to csv
with open("../data_output/works.csv", 'w', encoding='utf-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows(titles)

# Write "work_authors" to csv    
with open("../data_output/author_works.csv", 'w', encoding='utf-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows(work_authors)
    
with open("../data_output/work_subjects.csv", 'w', encoding='utf-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows(work_subject)    
    
with open("../data_output/subjects.csv", 'w', encoding='utf-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows(subject_tbl)   

### Step 2: Process 'authors' data
**Inputs:**   
../data_output/ai-authors.tsv

**Outputs:**  
../data_output/authors.csv

In [ ]:
doc = "../data_output/ai-authors.tsv"
df = pd.read_csv(doc, sep='\t', header=None)

# add column names
df.columns = ['type', 'path', 'revisions', 'timestamp', 'details']

# get 'work_id' from 'path'
df['author_id'] = df.path.str[9:]

# create a list of lists containing only the id and json data fields
authors = df[['author_id', 'details']].values.tolist()

author_ids = []  # initialize a list to hold extracted data
# iterate through the list and extract author names from the json data
for author in authors:
    details = json.loads(author[1])
    
    # append authour_id and name to list
    author_ids.append([author[0], details["name"]])

# write author_ids list to a csv file    
with open("../data_output/authors.csv", 'w', encoding='utf-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows(author_ids)
       

### Step 3: Process 'editions' data
**Inputs:**   
../data_output/ai-editions.tsv

**Outputs:**  


In [35]:
doc = "../data_output/ai-editions.tsv"
df = pd.read_csv(doc, sep='\t', header=None)

# add column names
df.columns = ['type', 'path', 'revisions', 'timestamp', 'details']

# get 'work_id' from 'path'
df['edition_id'] = df.path.str[7:]

# create a list of lists containing only the id and json data
editions_master = df[['edition_id', 'details']].values.tolist()

edition_work = [] # edition_id, work_id pairs
edition_pages = []
edition_isbn10 = []
edition_isbn13 = []
edition_title = []
edition_physical_format = []
edition_publisher = []

for edition in editions_master:
    details = json.loads(edition[1])
    
    # get work_ids
    edition_work.append([edition[0], details['works'][0]['key'][7:]])
    
    # get number of pages
    if 'number_of_pages' in details.keys():
        edition_pages.append([edition[0], details['number_of_pages']])
        
    # get isbn10
    if 'isbn_10' in details.keys():
        edition_isbn10.append([edition[0], details['isbn_10']])
    
    # get isbn13
    if 'isbn_13' in details.keys():
        edition_isbn13.append([edition[0], details['isbn_13']])

    # get title
    if 'title' in details.keys():
        edition_title.append([edition[0], details['title']])

    # get format
    if 'physical_format' in details.keys():
        edition_physical_format.append([edition[0], details['physical_format']])
        
    
    # get publisher data
    pubn = 0
    publisher_ids = {}
    if 'publishers' in details.keys():
        publisher = details['publishers'][0]
        if publisher not in publisher_ids.keys():
            publisher_ids[publisher] = str(n)
            n += 1
        if 'publish_places' in details.keys():
            pub_place = details['publish_places'][0]
        else:
            pub_place = "NULL"
        if 'publish_date' in details.keys():
            pub_date = details['publish_date']
        else:
            pub_date = "NULL"
        edition_publisher.append([publisher_ids[publisher], publisher, pub_place, pub_date])

#with open("../data_output/edition_work.csv", 'w', encoding='utf-8') as myfile:
#    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#    wr.writerows(edition_work)

### Process 'publisher' data

In [26]:
import csv
import pandas as pd
import json
doc = "../data_output/ai-editions.tsv"
df = pd.read_csv(doc, sep='\t', header=None)

# add column names
df.columns = ['type', 'path', 'revisions', 'timestamp', 'details']

# get 'work_id' from 'path'
df['edition_id'] = df.path.str[7:]

# create a list of lists containing only the id and json data
editions_master = df[['edition_id', 'details']].values.tolist()
edition_publisher = [["edition_id", "publisher_id"]]
publisher_place = [["pub_id", "place_id"]]
publisher_ids = {"NULL": "0"} # pub_name => pub_id
place_ids = {"NULL": "0"} # place_name => place_id

pub_id = 1
place_id = 1
for edition in editions_master:
    details = json.loads(edition[1])

    if 'publishers' in details.keys():
        publisher = details['publishers'][0]
        
        # create a unique id for publisher
        if publisher not in publisher_ids.keys():
            publisher_ids[publisher] = str(pub_id)
            pub_id += 1

        if 'publish_places' in details.keys():
            pub_place = details['publish_places'][0]
        else:
            pub_place = "NULL"
        
        # create a unique id for publisher place
        if pub_place not in place_ids.keys():
            place_ids[pub_place] = str(place_id)
            place_id += 1
        
        # Move this to edition_tbl
        #if 'publish_date' in details.keys():
        #    pub_date = details['publish_date']
        #else:
        #    pub_date = "NULL"
        
        edition_publisher.append([edition[0], publisher_ids[publisher]])
        publisher_place.append([publisher_ids[publisher], place_ids[pub_place]])
        
with open("../data_output/edition_publisher.csv", 'w', encoding='utf-8', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows(edition_publisher)
with open("../data_output/publisher_place.csv", 'w', encoding='utf-8', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows(publisher_place)
        
# publisher_ids and place_ids are dicts so they need to be handled differently
with open("../data_output/publishers.csv", 'w', encoding='utf-8', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(['pub_name', 'pub_id'])
    for key in publisher_ids.keys():
        wr.writerow([key, publisher_ids[key]])
        
with open("../data_output/places.csv", 'w', encoding='utf-8', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(['place_name', 'place_id'])
    for key in place_ids.keys():
        wr.writerow([key, place_ids[key]])

In [20]:
publisher_ids

{'NULL': '0', 'The MIT Press': '1'}

### Record viewer
Change n to look at individual json records.

In [24]:
n = 1
json.loads(editions_master[n][1])

{'publishers': ['Springer'],
 'physical_format': 'Hardcover',
 'first_sentence': {'type': '/type/text',
  'value': 'Conceptual Graphs are a flexible, extensible method for knowledge representation.'},
 'key': '/books/OL10325063M',
 'weight': '13.9 ounces',
 'title': 'Reasoning and Unification over Conceptual Graphs',
 'number_of_pages': 162,
 'isbn_13': ['9780306474873'],
 'covers': [2358367],
 'edition_name': '1 edition',
 'languages': [{'key': '/languages/eng'}],
 'isbn_10': ['0306474875'],
 'latest_revision': 3,
 'last_modified': {'type': '/type/datetime',
  'value': '2010-04-13T05:50:23.081550'},
 'authors': [{'key': '/authors/OL3419887A'}],
 'publish_date': 'January 31, 2003',
 'works': [{'key': '/works/OL9383183W'}],
 'type': {'key': '/type/edition'},
 'subjects': ['Artificial intelligence',
  'Computers',
  'Computers - General Information',
  'Computer Books: General',
  'Artificial Intelligence - General',
  'Discrete Mathematics',
  'Information Theory',
  'Computers / Artifi

In [22]:
csv_columns = ['No','Name','Country']
dict_data = [
{'No': 1, 'Name': 'Alex', 'Country': 'India'},
{'No': 2, 'Name': 'Ben', 'Country': 'USA'},
{'No': 3, 'Name': 'Shri Ram', 'Country': 'India'},
{'No': 4, 'Name': 'Smith', 'Country': 'USA'},
{'No': 5, 'Name': 'Yuva Raj', 'Country': 'India'},
]
csv_file = "Foobar.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:

SyntaxError: unexpected EOF while parsing (<ipython-input-22-4aea696d7d84>, line 16)